Importar las librerias

In [133]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils

from tensorflow.keras import Sequential

from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

Acceder a los datos

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
base_dir= '/content/gdrive/MyDrive/7mo semestre/Big Data/NLP_intento/train.csv'

In [ ]:
import pandas as pd
data=pd.read_csv(base_dir)

In [ ]:
data2=data[:10000]


In [ ]:
data2.columns = ['label', 'Titulo','Comentario']

In [ ]:
train_df=data2[:8000]
val_df=data2[8000:]

In [ ]:
print(train_df['label'].dtypes)

int64


In [ ]:
# Crear conjuntos de datos de TensorFlow a partir de DataFrames
batch_size = 50


train_data = tf.data.Dataset.from_tensor_slices((train_df['Titulo'].values, train_df['label'].values))
train_data = train_data.batch(batch_size)

val_data= tf.data.Dataset.from_tensor_slices((val_df['Titulo'].values, val_df['label'].values))
val_data = val_data.batch(batch_size)


In [ ]:
test_df=data[10000:12000]
test_df.columns = ['label', 'Titulo','Comentario']
test_data= tf.data.Dataset.from_tensor_slices((test_df['Titulo'].values, test_df['label'].values))
test_data = test_data.batch(batch_size)

In [ ]:
# Realizar el recuento de datos por clase
conteo_por_clase = test_df["label"].value_counts()
print('test:\n',conteo_por_clase)
conteo_por_clase = train_df["label"].value_counts()
print('train:\n',conteo_por_clase)
conteo_por_clase = val_df["label"].value_counts()
print('validation:\n',conteo_por_clase)

test:
 1    1035
2     965
Name: label, dtype: int64
train:
 1    4097
2    3903
Name: label, dtype: int64
validation:
 1    1001
2     999
Name: label, dtype: int64


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
for text_batch, label_batch in train_data.take(1):
  for i in range(10):
    print("comentario: ", text_batch.numpy()[i])
    print("Label:", label_batch.numpy()[i])

comentario:  b'The best soundtrack ever to anything.'
Label: 2
comentario:  b'Amazing!'
Label: 2
comentario:  b'Excellent Soundtrack'
Label: 2
comentario:  b'Remember, Pull Your Jaw Off The Floor After Hearing it'
Label: 2
comentario:  b'an absolute masterpiece'
Label: 2
comentario:  b'Buyer beware'
Label: 1
comentario:  b'Glorious story'
Label: 2
comentario:  b'A FIVE STAR BOOK'
Label: 2
comentario:  b'Whispers of the Wicked Saints'
Label: 2
comentario:  b'The Worst!'
Label: 1


In [ ]:
VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 10

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

In [ ]:
# tomar solo el texto y adaptarlo al dataframe con  `TextVectorization.adapt`.
train_text = train_data.map(lambda text, labels: text)
int_vectorize_layer.adapt(train_text)

In [ ]:
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label

In [ ]:
# Retrieve a batch (of 32 reviews and labels) from the dataset.
text_batch, label_batch = next(iter(train_data))
first_question, first_label = text_batch[0], label_batch[0]
print("Comentario", first_question)
print("Label", first_label)

Comentario tf.Tensor(b'The best soundtrack ever to anything.', shape=(), dtype=string)
Label tf.Tensor(2, shape=(), dtype=int64)


In [ ]:
print("'int' vectorized question:",
      int_vectorize_text(first_question, first_label)[0])

'int' vectorized question: tf.Tensor([[  2  21 456  30  12 981   0   0   0   0]], shape=(1, 10), dtype=int64)


In [ ]:
print("1289 ---> ", int_vectorize_layer.get_vocabulary()[1289])
print("313 ---> ", int_vectorize_layer.get_vocabulary()[313])
print("Vocabulary size: {}".format(len(int_vectorize_layer.get_vocabulary())))

1289 --->  threes
313 --->  clan
Vocabulary size: 6328


vecrtorizar los datasets

In [ ]:
int_train_ds = train_data.map(int_vectorize_text)
int_val_ds = val_data.map(int_vectorize_text)
int_test_ds = test_data.map(int_vectorize_text)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
int_train_ds = configure_dataset(int_train_ds)
int_val_ds = configure_dataset(int_val_ds)
int_test_ds = configure_dataset(int_test_ds)

Modelo 1

In [ ]:
int_model = tf.keras.Sequential([
      layers.Embedding(6328, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(32, activation='relu')
      layers.Dense(16, activation='relu')
      layers.Dense(1, activation='sigmoid')])

int_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5820: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


160/160 [==============================] - 4s 13ms/step - loss: -25.4950 - accuracy: 0.5090 - val_loss: -105.5158 - val_accuracy: 0.5005
Epoch 2/10
160/160 [==============================] - 2s 12ms/step - loss: -443.4123 - accuracy: 0.5121 - val_loss: -993.1104 - val_accuracy: 0.5005
Epoch 3/10
160/160 [==============================] - 1s 9ms/step - loss: -2083.7180 - accuracy: 0.5121 - val_loss: -3535.2131 - val_accuracy: 0.5005
Epoch 4/10
160/160 [==============================] - 1s 8ms/step - loss: -5749.0356 - accuracy: 0.5121 - val_loss: -8466.0146 - val_accuracy: 0.5005
Epoch 5/10
160/160 [==============================] - 1s 8ms/step - loss: -12106.3184 - accuracy: 0.5121 - val_loss: -16394.3848 - val_accuracy: 0.5005
Epoch 6/10
160/160 [==============================] - 1s 8ms/step - loss: -21709.8789 - accuracy: 0.5121 - val_loss: -27827.3887 - val_accuracy: 0.5005
Epoch 7/10
160/160 [==============================] - 1s 7ms/step - loss: -35022.5117 - accuracy: 0.5121 - val

In [ ]:
print(int_model.summary())

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 64)          404992    
                                                                 
 conv1d_2 (Conv1D)           (None, None, 64)          20544     
                                                                 
 global_max_pooling1d_2 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 425601 (1.62 MB)
Trainable params: 425601 (1.62 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
int_loss, int_accuracy = int_model.evaluate(int_test_ds)
print("Int model accuracy: {:2.2%}".format(int_accuracy))

40/40 [==============================] - 0s 2ms/step - loss: -116849.9766 - accuracy: 0.5175
Int model accuracy: 51.75%


In [ ]:
export_model = tf.keras.Sequential(
    [int_vectorize_layer, int_model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

# Test it with `test_data`, which yields raw strings
loss, accuracy = export_model.evaluate(test_data)
print("Accuracy: {:2.2%}".format(accuracy))

40/40 [==============================] - 0s 2ms/step - loss: -0.1692 - accuracy: 0.5175
Accuracy: 51.75%


Salvamos el modelo

In [ ]:
int_model.save('int_modelo.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Modelo 2

In [ ]:
model = tf.keras.Sequential([
      layers.Embedding(6328, 64, mask_zero=True),
      #layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      #layers.GRU(256, return_sequences=True),
      layers.LSTM(128),
      layers.Dense(10),
      layers.Dense(1, activation='sigmoid')])


model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, None, 64)          404992    
                                                                 
 lstm_4 (LSTM)               (None, 128)               98816     
                                                                 
 dense_10 (Dense)            (None, 10)                1290      
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 505109 (1.93 MB)
Trainable params: 505109 (1.93 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(
    loss=losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = model.fit(int_train_ds, validation_data=int_val_ds, epochs=10)

Epoch 1/10
160/160 [==============================] - 16s 69ms/step - loss: -28.9954 - accuracy: 0.5098 - val_loss: -75.9229 - val_accuracy: 0.5005
Epoch 2/10
160/160 [==============================] - 8s 52ms/step - loss: -140.4682 - accuracy: 0.5121 - val_loss: -222.6075 - val_accuracy: 0.5005
Epoch 3/10
160/160 [==============================] - 7s 41ms/step - loss: -322.2827 - accuracy: 0.5121 - val_loss: -439.2800 - val_accuracy: 0.5005
Epoch 4/10
160/160 [==============================] - 7s 45ms/step - loss: -575.6989 - accuracy: 0.5121 - val_loss: -724.2499 - val_accuracy: 0.5005
Epoch 5/10
160/160 [==============================] - 5s 31ms/step - loss: -895.4985 - accuracy: 0.5121 - val_loss: -1073.0289 - val_accuracy: 0.5005
Epoch 6/10
160/160 [==============================] - 7s 47ms/step - loss: -1276.3673 - accuracy: 0.5121 - val_loss: -1482.3811 - val_accuracy: 0.5005
Epoch 7/10
160/160 [==============================] - 5s 33ms/step - loss: -1716.0739 - accuracy: 0.5121

Cargamos el modelo 1, ya que ambos tuvieron el mismo accuracy

In [ ]:
loaded_model = tf.keras.models.load_model('int_modelo.h5')

In [ ]:
pruebas=data[12010:12020]
pruebas.columns = ['label', 'Titulo','Comentario']

In [ ]:
pruebas

,label,Titulo,Comentario
12010,2,The Bible Made Easy: A Book-by-book Introduction,A quick and easy way to get to know the Bible....
12011,1,This book appeared to a small group of shepher...,Unlike other religiously oriented books such a...
12012,2,Is the Pope Catholic?,The Third Secret is a timely piece of fiction ...
12013,1,Sigh,I wish the Catholic bashing would stop; perhap...
12014,1,"A disappointment and, in the end, a waste of time","I kept reading past the midpoint of the book, ..."
12015,1,Don't waste your money,This book is rife with misconceptions about th...
12016,1,Couldn't Finish It...,I began reading Steve Berry's books in order. ...
12017,1,I'm weary of Berry's re-writes of Christianity,I've now read all of Berry's books and althoug...
12018,2,The Third Secret,"Awaiting the new Dan Brown novel, left me in s..."
12019,2,Fans of Da Vinci Code will enjoy...,"Let's face it, most of these books are getting..."


Probamos el modelo

In [ ]:
#Aplicar la vectorización de texto a los datos de entrada
inputs=pruebas['Titulo']
vectorized_inputs = int_vectorize_layer(inputs)

# Realizar predicciones con el modelo cargado
predicted_scores = loaded_model.predict(vectorized_inputs)

1/1 [==============================] - 0s 71ms/step


In [ ]:
import numpy as np

predicted_classes = [1 if score > 0.5 else 2 for score in np.argmax(predicted_scores, axis=1)]

In [ ]:
predicted_classes

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [ ]:
pruebas['label']

12010    2
12011    1
12012    2
12013    1
12014    1
12015    1
12016    1
12017    1
12018    2
12019    2
Name: label, dtype: int64